## Final Project Submission

Please fill out:
* Student name: Grace Jepchumba Rotich
* Student pace:  full time
* Scheduled project review date/time: 
* Instructor name: Diana 
* Blog post URL: 


Imported necessary libraries and modules.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import sqlite3

Using Panadas to read from the bom.movie_gross.csv file and check out the columns to understand the data.

In [23]:
bom_movie = pd.read_csv('Data/bom.movie_gross.csv')
bom_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


This provides an overview of the DataFrame's structure, including the number of entries, columns, their names, data types, and memory usage. It's helpful for understanding the dataset's composition and characteristics at a glance.


Inorder to access the imdb file which is a sql database, a connection is created using the Sqlite3 module. Also utilizes the pd.read_sql() function from the pandas library to execute a SQL query against an SQLite database. 


In [38]:
conn = sqlite3.connect('./Data/im.db')
pd.read_sql("""
            SELECT name 
            FROM sqlite_master 
            WHERE type='table';
""", conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


# **Data Presentation**



<img src="Images\datapresentation.png" alt="in image folder" style="width:800px;height:400px;">


In [39]:
bom_movie.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


The dataframe has 5 columns and 3387 rows of entries. There are some missing values in the studio, domestic gross and foreign gross columns.
Since the null values in the studio columns is few, lets see if we can drop them. First we filter the DataFrame bom_movie to include only those rows where the 'studio' column has missing values (NaN).

In [40]:
bom_movie.loc[bom_movie['studio'].isna() == True]

,title,studio,domestic_gross,foreign_gross,year
210,Outside the Law (Hors-la-loi),NaN,96900.0,3300000,2010
555,Fireflies in the Garden,NaN,70600.0,3300000,2011
933,Keith Lemon: The Film,NaN,NaN,4000000,2012
1862,Plot for Peace,NaN,7100.0,NaN,2014
2825,Secret Superstar,NaN,NaN,122000000,2017


The above movies can be dropped since they are missing values and have made litte gross profits compaired to the rest of the movies in the data set.

In [41]:
bom_movie = bom_movie.dropna(subset=['studio'])

Lets checkout the top movies based off of their foreign_gross and domestic gross group them by the studio that produced them but foreign_gross is in string format so we should first convert to a float.We would strip all the commas from the strings in foreign gross and then convert them to float and replace the values with the new float values this will make it easier when using aggregate functions

In [42]:
bom_movie['foreign_gross'] = bom_movie['foreign_gross'].str.replace(',','')
bom_movie['foreign_gross'] = pd.to_numeric(bom_movie['foreign_gross'])

In [43]:
bom_movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3382 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3382 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3356 non-null   float64
 3   foreign_gross   2033 non-null   float64
 4   year            3382 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 158.5+ KB


After fixing up the data type of the foreign gross column the next step would be to sort out the values to pick out the top ten movies based off of foreign gross that the movie earned and assigned it to a dataframe called top_10_movies_revenue

In [44]:
top_10_movies_revenue = bom_movie.sort_values(by=['foreign_gross','domestic_gross'], ascending=[False,False]).head(10)
top_10_movies_revenue

,title,studio,domestic_gross,foreign_gross,year
328,Harry Potter and the Deathly Hallows Part 2,WB,381000000.0,960500000.0,2011
1875,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012
3081,Jurassic World: Fallen Kingdom,Uni.,417700000.0,891800000.0,2018
1127,Frozen,BV,400700000.0,875700000.0,2013
2764,Wolf Warrior 2,HC,2700000.0,867600000.0,2017
1477,Transformers: Age of Extinction,Par.,245400000.0,858600000.0,2014
1876,Minions,Uni.,336000000.0,823400000.0,2015
3083,Aquaman,WB,335100000.0,812700000.0,2018
1128,Iron Man 3,BV,409000000.0,805800000.0,2013


Created a dataframe called movie_info for the movie_info tsv file and observed the first 5 rows it contains to see if the data could be used for the project

In [45]:
movie_info = pd.read_csv('Data/rt.movie_info.tsv', delimiter='\t')
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


Created a dataframe called tmovies for tmdb.movies.csv file and observed the first 5 rows it contains to see if the data could be used for the project

In [46]:
tmovies = pd.read_csv('Data/tmdb.movies.csv')
tmovies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


Lets inspect the tmovies dataframe

In [47]:
tmovies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


There are 26517 entries and there are no missing values in the entire dataframe. Hence, w can use it without a lot of cleaning.
Sorting the dataframe using the vote_average , vote_count and popularity column.

In [48]:
tmovies = tmovies.sort_values(by=['vote_average','vote_count','popularity'], ascending=[False,False,False])
tmovies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
3687,3687,"[35, 14, 18, 10749]",307124,en,Love on a Leash,1.400,2013-08-13,Love on a Leash,10.0,7
9199,9199,"[35, 14, 18, 10749]",307124,en,Love on a Leash,1.400,2013-08-13,Love on a Leash,10.0,7
19174,19174,[10402],411990,en,Endless,1.003,2016-08-19,Endless,10.0,4
12742,12742,[99],276913,en,Hidden Colors 3: The Rules of Racism,0.968,2014-06-26,Hidden Colors 3: The Rules of Racism,10.0,4
24326,24326,"[16, 12]",561974,ko,점박이 한반도의 공룡 2: 새로운 낙원,7.941,2018-12-31,Dino King 3D: Journey to Fire Mountain,10.0,3


AS shown above, there are some duplicates to be dropped for better data analysis. Also dropped the unnamed and genre_ids columns since its data cannot be understood.

In [ ]:
#dropping the unnamed and genre_id columns
tmovies = tmovies.drop(['Unnamed: 0','genre_ids'], axis=1)
#dropping any duplicated